In [2]:
!pip install requests
!pip install BeautifulSoup4
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re


In [3]:
#URL取得、はれるやより
result = requests.get("http://www.hareruyamtg.com/purchase/e/eBuy-Mdn_sspd/")
soup = BeautifulSoup(result.text, "lxml")

In [4]:
tag_a = soup.find_all('a')

In [7]:
for x in tag_a:
    tmp = x.get('href')
    tmp = re.findall(r'purchase/g/g',tmp)
    if tmp:
        print(x.get('href'))

/purchase/g/gISD000187EN/
/purchase/g/gSOM000121EN/
/purchase/g/gWWK000051EN/
/purchase/g/gMM3000076EN/
/purchase/g/gMM2015000223EN/
/purchase/g/gAVR000225EN/
/purchase/g/gNPH000016EN/
/purchase/g/gZEN000124EN/
/purchase/g/gFUT000140EN/
/purchase/g/gMM3000232EN/
/purchase/g/gMM3000244EN/
/purchase/g/gMM2015000004EN/
/purchase/g/gCON000041EN/
/purchase/g/gMM3000050EN/
/purchase/g/gMM2015000151EN/
/purchase/g/gISD000135EN/
/purchase/g/gMM3000141EN/
/purchase/g/gMM2015000165EN/
/purchase/g/gMMA000057EN/
/purchase/g/gFUT000061EN/
/purchase/g/gCSP000031EN/
/purchase/g/gRAV000050EN/
/purchase/g/g5DN000021EN/
/purchase/g/gMRD000193EN/
/purchase/g/gMMA000064EN/
/purchase/g/gMMA000202EN/
/purchase/g/gZEN000184EN/
/purchase/g/gMM3000249EN/
/purchase/g/gROE000026EN/
/purchase/g/gMM2015000003EN/
/purchase/g/gMM2015000075EN/
/purchase/g/gMMA000066EN/
/purchase/g/gZEN000141EN/
/purchase/g/gMM3000240EN/
/purchase/g/gSOM000171EN/
/purchase/g/gWWK000004EN/
/purchase/g/gMOR000039EN/
/purchase/g/gCHK0002

In [6]:
se_cardlist = pd.Series(card_name_list)
se_cardlist[se_cardlist.str.replace("【ENG】《", "", regex = True)]

NameError: name 'card_name_list' is not defined